<p style="text-align: center;"><font size="8"><b>Good Software Practices</b></font><br>


The last few lectures have introduced classes. You should by now be somewhat confident in designing and writing simple classes. With a little more thought, these concepts can be extended to design larger and more complicated programs. 

Larger programs require more time spent on design, testing and documentation. This lecture will cover these topics by going over how to design and implement a simple Tic-Tac-Toe game. It follows loosely chapter 7 in Goldwasser and Letscher. 

# Tic-Tac-Toe

You probably know the rules of Tic-Tac-Toe, but just to clarify:

1. Starting with an empty 3x3 board, a player marks a cell with an "x"
2. The computer then marks an empty cell with an "o"
3. The player marks an empty cell with an "x"
4. This process repeats until we have a line of either three "x"s or three "o"s.

# Top-down design

The tempation now is to start coding right away. This temptation should be avoided. It is better to spend a few minutes (or hours/days/months depending on the size of the problem) deciding the best way to implement a program. 



The first step is to envision the final project. In our case this mean deciding what the final Tic-Tac-Toe game will look like. This will be the top-level class. From there we expand the design to include classes that the top level class will need. This is known as **top-down design**.

For example, let's say we want our top level class to be called `TicTacToe`. This class might have a method `play()` that starts the game. 

In [ ]:
game = TicTacToe()
game.play()

What happens next?

We'll make a game that is played entirely on the console. So after we start the game, perhaps we display a blank board and ask the player for a move. To to this, the player will have to specify both a row and column, and we will have to read this in somehow and make sure it is a valid move.

    - - -
    - - -
    - - -
    
    Select row:
    Select column:

Next we will have to update the board. The player's move should be recorded and the computer should make a move somehow. Assume the player marks their moves with an x and the computer with and o. 

We should then print the board with the player and computer moves and ask for another move from the player.


    - x -
    - - o
    - - -
    
    Select row:
    Select column:

We'll keep repeating this until either there is a winner, or the board is full and the game is a tie.

    - x -
    - x o
    o x -
    
    PLAYER WINS!

So that's the top level class. *How* it works has yet to be determined, but at least we know *what* it should do. 

To actually implement it we will need to store and manipulate the board. To do this we could use a dedicated `Board` class. 

What should the `Board` class do?

For starters it will need to store the board, as well as any player and computer moves. It will also have to be able to print the board to the console. Being able to print to the console means we should be able to pass a Board into the `print` command. This means our Board class will have to implement the `__str__` special method.

It will also need to verify that a move is valid and add it to the Board. There are two types of moves: player and computer. The player specifies exactly what their move is, while the computer move must be determined somehow. 

Finally, the Board class must be able to determine if the game is over. In other words after every move, check if:
* the player has won
* the compuer has won
* the board is full (tie)

We can represent these two classes with UML diagrams.

![tic tac toe UML](https://github.com/lukasbystricky/ISC-3313/blob/master/lectures/chapter7/images/tic-tac-toe_UML.png?raw=true)

# Bottom-up Implementation

This completes the design stage. Now we actually have to implement these classes. Implementation typically starts at the lowest level. This is known as **bottom-up implementation**. For this to work properly, each component must be tested thouroughly before moving to the next level. 

For us, our lowest level is the `Board` class.

## Board class

The Board class must contain some sort of way to represent the state of the board. One way (not the only way) is to use a 2D NumPy array. For example the board 
   
    x - - 
    - o -
    - x o

could be represented by:

    np.array([["x","-","-"], ["-","o","-"], ["-","x","o"]])

An empty board would be represented by:

    np.array([["-","-","-"],["-","-","-"],["-","-","-"]])

So we are ready to start creating the Board class. The `__init__` routine should set the `board` member to be an empty board.

In [ ]:
import numpy as np

class Board:
    
    def __init__(self):
         self.board = np.array([["-","-","-"],["-","-","-"],["-","-","-"]])

Next we will want a method to convert the board to a string that can be printed to the console. This of course is the `__str__` method. Numpy arrays can already be converted to strings.

In [17]:
a = np.array([["-","-","-"],["-","-","-"],["-","-","-"]])
print(a)

[['-' '-' '-']
 ['-' '-' '-']
 ['-' '-' '-']]


This is close to what we want, but we will have to clean it up a bit.

In [19]:
class Board:
    
    def __init__(self):
        self._board = np.array([["-","-","-"],["-","-","-"],["-","-","-"]])
        
    def __str__(self):
        b_string = str(self._board)
        b_string = b_string.replace("[","")
        b_string = b_string.replace("]","")
        b_string = b_string.replace("\'","")
        b_string = b_string.replace("\n ","\n")
        return b_string

At this point we can already begin to test our Board class by printing the empty board to the console.

In [18]:
b = Board()
print(b)

- - -
- - -
- - -


Seems to work well enough so far. This is a nice feature of the bottom-up implementation. We can test every function we write as soon as we write it. This will lead to more robust code down the road.

The next method we should implement is a method to test if the game has been won. Remember what it means for a game to be won. A game has been if any of the following are true:

1. any of the diagonals are all the same character (either "x" or "o")
2. any of the rows are all the same character
3. any of the columns are all the same character

We will write a function `check_winner()` that takes in as a parameter a character (either "x" or "o") and checks to see if that that character qualifies as a winner.

We will make use of the `np.all()` function. The `np.all()` function takes in an array of booleans and returns True if they are all True and False otherwise. Our `check_winner()` function might thus look like:

In [ ]:
def check_winner(self, character):

    winner = False
    # find all entries on the board that are the character
    boardTmp = self._board == character

    # Check diagonals
    d1 = np.array([boardTmp[0,0], boardTmp[1,1], boardTmp[2,2]])
    d2 = np.array([boardTmp[0,2], boardTmp[1,1], boardTmp[2,0]])

    if np.all(d1) or np.all(d2):
            winner = True

    # Check rows and columns 
    if not(winner):
        for i in range(3):

            if np.all(boardTmp[i,:]) or np.all(boardTmp[:,i]):
                winner = True
                break

    return winner

Adding this method to the Board class:

In [20]:
class Board:
    
    def __init__(self):
        self._board = np.array([["-","-","-"],["-","-","-"],["-","-","-"]])
        
    def __str__(self):
        b_string = str(self._board)
        b_string = b_string.replace("[","")
        b_string = b_string.replace("]","")
        b_string = b_string.replace("\'","")
        b_string = b_string.replace("\n ","\n")
        return b_string
    
    def check_winner(self, character):

        winner = False
        # find all entries on the board that are the character
        boardTmp = self._board == character

        # Check diagonals
        d1 = np.array([boardTmp[0,0], boardTmp[1,1], boardTmp[2,2]])
        d2 = np.array([boardTmp[0,2], boardTmp[1,1], boardTmp[2,0]])

        if np.all(d1) or np.all(d2):
                winner = True

        # Check rows and columns 
        if not(winner):
            for i in range(3):

                if np.all(boardTmp[i,:]) or np.all(boardTmp[:,i]):
                    winner = True
                    break

        return winner

To test our winner method, we can modify the `_board` member.

In [21]:
b = Board()

print(b.check_winner("x"))
b._board = np.array([["x", "x", "x"],[ "-", "-", "-"], ["o", "x", "o"]])
print(b.check_winner("o"))
print(b.check_winner("x"))

False
False
True


## Exercise

Create a file Board.py containing the `Board` class. Write a code snippet that tests our Board class to make sure `check_winner()` works for a Board with "x" down the diagonal.

We will also need to check if the board is full. If it is (and there is no winner), the game is a tie. 

This is actually not difficult. We will again make use of the `np.any()` function. First we will compare the `_board` member to "-" to create an array of booleans. Then we will check if this array is `True` anywhere. If it is not, then the board is not full.

In [10]:
def full_board(self):
        boardTmp = self._board == "-"
        return not(np.any(boardTmp))

## Exercise

Add the `full_board()` function to the `Board` class and test to make sure it works.

The next method we will need to a method to read in a player move and update the board. To do this we will make use of the `input()` function. 

The `input()` function takes in a string as an argument and displays it to the console. Typically this string is some sort of prompt. The user can then type in an answer and press enter. Whatever the user types is then returned by the `input()` function.

For example:

In [22]:
name = input("Please enter your name:")
print("Your name is", name)

Please enter your name:John
Your name is John


The idea is that we will use the input function to ask the user for both a row and a column to mark with the letter "x". The numbers the user enters must satisfy two conditions:

1. They must both be in the range 1 to 3. The user cannot enter -1 or 8 for either the row or column number for example
2. The cell the user selects must be empty

What do we do if the user enters an invalid cell? 

We first ask the user for a cell, then we check if it is valid. If it is, we mark the cell with an "x" and check if the game has been won. If it's not valid, we display an error message and ask again.

After the user enters a valid cell, it needs to check if the player has won or if the board is full. It will return two booleans indicating which, if any of these scenarios is true.

In [ ]:
def player_move(self):

    valid = False
    while not(valid):
        row = int(input("Select row: "))
        column = int(input("Select column: "))

        if row < 1 or row > 3 or column < 1 or column > 3:
            print("Row and column number must be between 1 and 3")

        elif self._board[row-1,column-1] != "-":
            print("Please select an empty cell")

        else:
            self._board[row-1,column-1] = "x"
            valid = True

    return self.check_winner("x"), self.full_board()

## Excerise 

Add the `player_move()` function to the `Board` class and test it.

The final function needed for the `Board` class is the computer move function. 

We'll assume that the computer has no particular strategy and is just picking cells at random. Of course, the cell has to by empty. As with the player move, we will use a while loop to keep picking cells until we pick a valid one.

In [15]:
def computer_move(self):

    valid = False
    while not(valid):
        row = np.random.randint(0,3)
        column = np.random.randint(0,3)

        if self._board[row,column] == "-":
            self._board[row,column] = "o"
            valid = True

    return self.check_winner("o"), self.full_board()

## Exercise

Add the `computer_move()` function to the `Board` class and test it.

## TicTacToe class

Having completed the `Board` class, we now turn to the highest level class, the `TicTacToe` class.

Recall that the `TicTacToe` class has a single member variable of type `Board`. We will create this object in the `__init__` function.

In [16]:
class TicTacToe:
    
    def __init__(self):
        self._board = Board() # creates a blank Board
        
    

There is a single member function that must be added to the `TicTacToe` class. This is the `play()` method. 

As long as the game is not over (player or computer wins or tie), the `play()` method will print the board and ask the player for a move. Then provided the game is not over it will ask for a computer move. When the game is over a message will be displayed the gives the outcome of the game.


In [ ]:
def play(self):
    
    keep_playing = True
    while (keep_playing):
        print(str(self._board))
        [winner_player, full_board] = self._board.player_move()
        keep_playing = not(winner_player or full_board)

        if keep_playing:
            [winner_computer, full_board] = self._board.computer_move()                     
            keep_playing = not(winner_computer or full_board)

    print(str(self._board))

    if winner_player:
        print("Player wins!")

    elif winner_computer:
        print("Computer_wins!")

    else:
        print("Tie!")
            

## Exercise

Create a file TicTacToe.py containing the code for the `TicTacToe` class (don't forget to import the `Board` class). Add the `play()` method to the class and test to make sure the program works as expected.